# **Data Acquisition II**

## **Ajuste de decimais e encoding**

### **Decimais**

```
df  = pd.read_csv("/content/decimals.txt",
                sep=";", 
                decimal=",", 
                thousands=".", 
                index_col=False)
df
```

```
ANO 	ID_SEG 	COD_UF 	RENDA 	MES 	SB_CONS 	SB_ESCO 	SB_TRAT 	SB_U 	SB_G 	POP 	V10 	V11 	V12 	URG
0 	2012 	4 	10 	19712 	4 	28 	164 	3 	16 	0 	5274 	0.530906 	3.109594 	0.107143 	0.303375
1 	2012 	4 	10 	2547 	5 	13 	0 	6 	23 	0 	5274 	0.246492 	0.000000 	0.461538 	0.436102
2 	2012 	4 	10 	54598 	6 	8 	135 	7 	12 	0 	5274 	0.151688 	2.559727 	0.875000 	0.227531
3 	2012 	4 	10 	20712 	7 	0 	0 	0 	0 	0 	5274 	0.000000 	0.000000 	NaN 	0.000000
```


```df.dtypes```

```
NO          int64
ID_SEG       int64
COD_UF       int64
RENDA        int64
MES          int64
SB_CONS      int64
SB_ESCO      int64
SB_TRAT      int64
SB_U         int64
SB_G         int64
POP          int64
V10        float64
V11        float64
V12        float64
URG        float64
dtype: object
```

### **Encoding**

**Encoding** é uma técnica que define regras para armazenar os code points dos caracteres que compõem as _string_ na memória do computador. Existem vários deles: _UTF-8_, _ISO-8859-1_, _UCS-2_, _windows-1252_ e assim por diante.
```
encoding = 'latin1'
encoding = 'ISO-8859-1'
encoding = 'cp1251'
encoding = 'utf-8'
```

## **Conexão a modelos relacionais SQL usando Pandas**

### **Revisão sobre banco de dados**
#### **Os bancos de dados relacionais**
* Organizam dados sobre entidades em tabelas.
* Representam os dados como **um conjunto de tabelas em duas dimensões, formadas por linhas e colunas**. Cada linha ou registro é uma instância de uma entidade. Cada coluna tem a informação de um atributo.
* Frequentemente, os **bancos de dados relacionais contém várias tabelas relacionadas entre si**, o que permite que a informação seja armazenada de forma mais compacta.
* Suportam uma grande quantidade de dados, vários usuários simultaneamente e controles de qualidade dos dados.
* Os tipos dos dados estão especificados em cada coluna.
* Usamos o **SQL** (Structured Query Language) para interagir com o banco de dados. 

#### **Como acessar bancos de dados relacionais através do Pandas**
##### **Bibliotecas para acessar o DataFrama**

Carregar dados a um DataFrame pode ser bem simples com o uso das seguintes bibliotecas:
1. Sqlalchemy
2. Sqllite3
3. cx_Oracle
4. Pyodbc
5. etc

##### **SQLITE3**
###### **Uso de sqlite3 e Pandas**

* Com pandas, é muito simples combinar o motor particular usado e transformá-lo em um DataFrame usando SQL como string.
* Criamos a conexão com sqlite3 e, junto com uma query sql, a usamos de argumento com o método read_sql_query do Pandas.
```
con = sqlite3.connect(base_sql)
df = pd.read_sql_query(sql, con)
```
1. Criamos a query SQL como string para extrair todas as tabelas do banco de dados.
2. Geramos a conexão ao banco **nba_salary.sqlite**.
3. Integramos o dataframe com a query e a conexão criada anteriormente.
4. Fazemos um print do DataFrame (Nesse caso, tem apenas dois elementos) e se encerra a conexão.

In [39]:
import os
from dotenv import load_dotenv
import sqlite3
import pandas as pd

In [58]:
load_dotenv()

caminho = os.getenv("DIRETORIO")
nome_arquivo = 'nba_salary.sqlite'

arquivo = os.path.join(caminho, nome_arquivo)

sql = "SELECT name FROM sqlite_master WHERE type='table'"
con = sqlite3.connect(arquivo)
df = pd.read_sql_query(sql, con)
print(df)

con.close()

                    name
0  NBA_season1718_salary
1          Seasons_Stats


* Através da query, também podemos extrair a tabela **NBA_season1718_salary:**

In [60]:
con = sqlite3.connect(arquivo)
df = pd.read_sql_query("SELECT * FROM NBA_season1718_salary", con)
df.head()

,X1,Player,Tm,season17_18
0,1.0,Stephen Curry,GSW,34682550.0
1,2.0,LeBron James,CLE,33285709.0
2,3.0,Paul Millsap,DEN,31269231.0
3,4.0,Gordon Hayward,BOS,29727900.0
4,5.0,Blake Griffin,DET,29512900.0


In [63]:
df.groupby('Tm')['Player'].count()

Tm
ATL    27
BOS    17
BRK    18
CHI    24
CHO    14
CLE    14
DAL    22
DEN    16
DET    20
GSW    16
HOU    22
IND    19
LAC    20
LAL    22
MEM    20
MIA    18
MIL    22
MIN    16
NOP    21
NYK    21
OKC    15
ORL    17
PHI    20
PHO    21
POR    19
SAC    20
SAS    17
TOR    17
UTA    20
WAS    18
Name: Player, dtype: int64

In [64]:
con.close()

##### **SqlAlchemy**

sqlalchemy pode ler qualquer tipo de banco de dados. É possível trocar o motor onde está escrito 'sqlite'.

In [65]:
import sqlalchemy as db

In [69]:
engine = db.create_engine('sqlite:///' + arquivo)
connection = engine.connect()

In [70]:
df = pd.read_sql_query('SELECT * FROM Seasons_Stats', connection)
df.head()

,X1,Year,Player,Pos,Age,Tm,G,GS,MP,PER,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,0.0,1950.0,Curly Armstrong,G-F,31.0,FTW,63.0,NaN,NaN,NaN,...,0.705,NaN,NaN,NaN,176.0,NaN,NaN,NaN,217.0,458.0
1,1.0,1950.0,Cliff Barker,SG,29.0,INO,49.0,NaN,NaN,NaN,...,0.708,NaN,NaN,NaN,109.0,NaN,NaN,NaN,99.0,279.0
2,2.0,1950.0,Leo Barnhorst,SF,25.0,CHS,67.0,NaN,NaN,NaN,...,0.698,NaN,NaN,NaN,140.0,NaN,NaN,NaN,192.0,438.0
3,3.0,1950.0,Ed Bartels,F,24.0,TOT,15.0,NaN,NaN,NaN,...,0.559,NaN,NaN,NaN,20.0,NaN,NaN,NaN,29.0,63.0
4,4.0,1950.0,Ed Bartels,F,24.0,DNN,13.0,NaN,NaN,NaN,...,0.548,NaN,NaN,NaN,20.0,NaN,NaN,NaN,27.0,59.0
